# Step 2: Data Cleaning & Preprocessing

**Objective:** Clean the raw dataset, engineer features, encode categoricals,
scale numericals, and produce train/test splits ready for modeling.

---

## 2.1 Setup

In [1]:
import sys
sys.path.insert(0, "..")

import pandas as pd
import numpy as np

from src.data_loader import load_dataset
from src.preprocessing import (
    drop_irrelevant_columns,
    handle_missing_values,
    fix_invalid_entries,
    validate_dtypes,
)
from src.feature_engineering import (
    create_date_features,
    encode_categoricals,
    scale_numericals,
    split_data,
)

df_raw = load_dataset("../data/raw/Flight_Price_Dataset_of_Bangladesh.csv")
print(f"\nRaw shape: {df_raw.shape}")

Loaded 57,000 rows × 17 columns from Flight_Price_Dataset_of_Bangladesh.csv

Raw shape: (57000, 17)


## 2.2 Drop Irrelevant Columns

In [2]:
df = drop_irrelevant_columns(df_raw.copy())
print(f"Shape after dropping: {df.shape}")

Renamed columns: {'Base Fare (BDT)': 'Base Fare', 'Tax & Surcharge (BDT)': 'Tax & Surcharge', 'Total Fare (BDT)': 'Total Fare', 'Departure Date & Time': 'Date', 'Duration (hrs)': 'Duration', 'Days Before Departure': 'DaysBeforeDeparture'}
Dropped redundant columns: ['Source Name', 'Destination Name', 'Arrival Date & Time']
No extra irrelevant columns found to drop.
Shape after dropping: (57000, 14)


## 2.3 Handle Missing Values

In [3]:
print("Before imputation:")
print(df.isnull().sum()[df.isnull().sum() > 0])

df = handle_missing_values(df)

print("\nAfter imputation:")
print(df.isnull().sum().sum(), "total missing")

Before imputation:
Series([], dtype: int64)
Total remaining missing values: 0

After imputation:
0 total missing


## 2.4 Correct Invalid Entries

In [4]:
df = fix_invalid_entries(df)

Rows: 57,000 → 57,000 (removed 0)


## 2.5 Validate & Convert Data Types

In [5]:
df = validate_dtypes(df)
print("\nFinal dtypes:")
print(df.dtypes)

  Base Fare → float64
  Tax & Surcharge → float64
  Total Fare → float64
  Date → datetime64
  Airline → category
  Source → category
  Destination → category
  Stopovers → category
  Aircraft Type → category
  Class → category
  Booking Source → category
  Seasonality → category

Final dtypes:
Airline                      category
Source                       category
Destination                  category
Date                   datetime64[ns]
Duration                      float64
Stopovers                    category
Aircraft Type                category
Class                        category
Booking Source               category
Base Fare                     float64
Tax & Surcharge               float64
Total Fare                    float64
Seasonality                  category
DaysBeforeDeparture             int64
dtype: object


/app/notebooks/../src/preprocessing.py:210: UserWarning: Parsing dates in %Y-%m-%d %H:%M:%S format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df[col] = pd.to_datetime(df[col], errors="coerce", dayfirst=True)


## 2.6 Feature Engineering

In [6]:
# Extract temporal features from Date
df = create_date_features(df)
df.head()

Created date features: Month, Day, Weekday, WeekdayName, Season


,Airline,Source,Destination,Date,Duration,Stopovers,Aircraft Type,Class,Booking Source,Base Fare,Tax & Surcharge,Total Fare,Seasonality,DaysBeforeDeparture,Month,Day,Weekday,WeekdayName,Season
0,Malaysian Airlines,CXB,CCU,2025-11-17 06:25:00,1.219526,Direct,Airbus A320,Economy,Online Website,21131.225021,5169.683753,26300.908775,Regular,10,11,17,0,Monday,Autumn
1,Cathay Pacific,BZL,CGP,2025-03-16 00:17:00,0.608638,Direct,Airbus A320,First Class,Travel Agency,11605.395471,200.000000,11805.395471,Regular,14,3,16,6,Sunday,Summer
2,British Airways,ZYL,KUL,2025-12-13 12:03:00,2.689651,1 Stop,Boeing 787,Economy,Travel Agency,39882.499349,11982.374902,51864.874251,Winter Holidays,83,12,13,5,Saturday,Winter
3,Singapore Airlines,RJH,DAC,2025-05-30 03:21:00,0.686054,Direct,Airbus A320,Economy,Direct Booking,4435.607340,200.000000,4635.607340,Regular,56,5,30,4,Friday,Summer
4,British Airways,SPD,YYZ,2025-04-25 09:14:00,14.055609,1 Stop,Airbus A350,Business,Direct Booking,59243.806146,14886.570922,74130.377068,Regular,90,4,25,4,Friday,Summer


In [7]:
# Encode categorical variables
df_encoded = encode_categoricals(df.copy())
print(f"Shape after encoding: {df_encoded.shape}")

One-hot encoded: ['Airline', 'Source', 'Destination', 'Stopovers', 'Aircraft Type', 'Class', 'Booking Source', 'Seasonality', 'WeekdayName', 'Season']  →  80 total columns
Shape after encoding: (57000, 80)


In [8]:
# Scale numerical features and persist the fitted scaler for inference
from src.feature_engineering import save_scaler

df_scaled, scaler = scale_numericals(df_encoded.copy())

scale_cols = [
    c for c in df_encoded.columns
    if any(k in c.lower() for k in ("fare", "tax", "surcharge"))
    and "total" not in c.lower()
]
save_scaler(scaler, scale_cols)

df_scaled.head()

Fit & transformed: ['Base Fare', 'Tax & Surcharge']
Saved scaler → data/processed/scaler.joblib  (columns: ['Base Fare', 'Tax & Surcharge'])


,Date,Duration,Base Fare,Tax & Surcharge,Total Fare,DaysBeforeDeparture,Month,Day,Weekday,Airline_Air Astra,...,Seasonality_Winter Holidays,WeekdayName_Monday,WeekdayName_Saturday,WeekdayName_Sunday,WeekdayName_Thursday,WeekdayName_Tuesday,WeekdayName_Wednesday,Season_Monsoon,Season_Summer,Season_Winter
0,2025-11-17 06:25:00,1.219526,-0.548639,-0.517851,26300.908775,10,11,17,0,0,...,0,1,0,0,0,0,0,0,0,0
1,2025-03-16 00:17:00,0.608638,-0.687016,-0.927748,11805.395471,14,3,16,6,0,...,0,0,0,1,0,0,0,0,1,0
2,2025-12-13 12:03:00,2.689651,-0.276250,0.044055,51864.874251,83,12,13,5,0,...,1,0,1,0,0,0,0,0,0,1
3,2025-05-30 03:21:00,0.686054,-0.791167,-0.927748,4635.607340,56,5,30,4,0,...,0,0,0,0,0,0,0,0,1,0
4,2025-04-25 09:14:00,14.055609,0.005001,0.283592,74130.377068,90,4,25,4,0,...,0,0,0,0,0,0,0,0,1,0


## 2.7 Train/Test Split

In [9]:
X_train, X_test, y_train, y_test = split_data(
    df_scaled, target="Total Fare", test_size=0.20, random_state=42
)
print(f"\nX_train: {X_train.shape}")
print(f"X_test:  {X_test.shape}")
print(f"y_train: {y_train.shape}")
print(f"y_test:  {y_test.shape}")

Dropped non-numeric columns before split: ['Date']
Train: 45,600 samples  |  Test: 11,400 samples
Saved 78 training column names → data/processed/train_columns.json
Saved splits to data/processed/

X_train: (45600, 78)
X_test:  (11400, 78)
y_train: (45600,)
y_test:  (11400,)


## 2.8 Before / After Summary

| Metric | Before | After |
|--------|--------|-------|
| Rows | *[fill in]* | *[fill in]* |
| Columns | *[fill in]* | *[fill in]* |
| Missing values | *[fill in]* | 0 |
| Duplicate rows | *[fill in]* | 0 |

**Encoding decisions:** *[Justify why you chose One-Hot vs Label Encoding]*

**Scaling decisions:** *[Justify why StandardScaler was chosen]*

---

## Phase 2 Summary

*[Write a summary of all transformations applied, how many rows were lost,
and confirm the data is ready for EDA and modeling.]*